In [1]:
import os

# from the current `tute_1` directory, go back two levels to the `MAST30034` directory
output_relative_dir = '../data/landing/'

# check if it exists as it makedir will raise an error if it does exist
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)
    
# now, for each type of data set we will need, we will create the paths
for target_dir in ('pluto_data',): # taxi_zones should already exist
    if not os.path.exists(output_relative_dir + target_dir):
        os.makedirs(output_relative_dir + target_dir)

In [2]:
NUM_ROWS_PLUTO = 859012
MAX_ROWS_PER_CALL = 1000
RELEVANT_COLUMNS = ["latitude", "longitude","assesstot"]

In [3]:
import pandas as pd
from sodapy import Socrata
from tqdm import tqdm

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

for i in tqdm(range(0, NUM_ROWS_PLUTO, MAX_ROWS_PER_CALL)):
    # print(f"Downloading {i} to {i + MAX_ROWS_PER_CALL} of {NUM_ROWS_PLUTO}")
    results = client.get("64uk-42ks", limit=MAX_ROWS_PER_CALL, offset=i, select=",".join(RELEVANT_COLUMNS))
    results_df = pd.DataFrame.from_records(results)
    results_df.to_parquet(f"{output_relative_dir}pluto_data/pluto_data_{i}.parquet", index=False)

  0%|          | 0/860 [00:06<?, ?it/s]


KeyboardInterrupt: 